# Lab3 - Activity 2 - KNN Classification with Credit Data

### References: 
#### 1. Lab3.pdf
#### 2. knn - Doing Data Science (Book) and Lecture Video
#### 3. CSV files - https://cran.r-project.org/doc/manuals/R-data.pdf or http://rprogramming.net/read-csv-in-r/¶
#### 4. Normalization - http://vitalflux.com/data-science-scale-normalize-numeric-data-using-r/

#### Include Necessary Libraries 

In [35]:
#install.packages("FNN", dependencies = TRUE, repos='http://cran.rstudio.com/')
library("FNN")  #KNN
library(dplyr)
library("gdata")

#### Read Table from CSV

In [2]:
Credit_Data <- read.csv("german_credit_data1.csv")
Credit_Data2 <- read.csv("german_credit_data1.csv")

In [3]:
head(Credit_Data)
#summary(Credit_Data)

X.,Age,Sex,Job,Housing,Saving.accounts,Checking.account,Credit.amount,Duration,Purpose,Credit.Risks
0,67,male,2,own,NA,little,1169,6,radio/TV,1
1,22,female,2,own,little,moderate,5951,48,radio/TV,2
2,49,male,1,own,little,NA,2096,12,education,1
3,45,male,2,free,little,little,7882,42,furniture/equipment,1
4,53,male,2,free,little,little,4870,24,car,2
5,35,male,1,free,NA,NA,9055,36,education,1


In [4]:
Credit_Data <- na.omit(Credit_Data)

In [5]:
Credit_Data$sex[Credit_Data$Sex=="male"] <- 1
Credit_Data$sex[Credit_Data$Sex=="female"] <- 2
Credit_Data$housing[Credit_Data$Housing=="own"] <- 1
Credit_Data$housing[Credit_Data$Housing=="free"] <- 2
Credit_Data$housing[Credit_Data$Housing=="rent"] <- 3
Credit_Data$savings[Credit_Data$Saving.accounts=="little"] <- 1
Credit_Data$savings[Credit_Data$Saving.accounts=="moderate"] <- 2
Credit_Data$savings[Credit_Data$Saving.accounts=="quite rich"] <- 3
Credit_Data$savings[Credit_Data$Saving.accounts=="rich"] <- 4
Credit_Data$checkings[Credit_Data$Checking.account=="little"] <- 1
Credit_Data$checkings[Credit_Data$Checking.account=="moderate"] <- 2
Credit_Data$checkings[Credit_Data$Checking.account=="rich"] <- 3
# Credit_Data$duration[Credit_Data$Duration>0 & Credit_Data$Duration<=12] <- 1
# Credit_Data$duration[Credit_Data$Duration>12 & Credit_Data$Duration<=24] <- 2
# Credit_Data$duration[Credit_Data$Duration>24 & Credit_Data$Duration<=36] <- 3
# Credit_Data$duration[Credit_Data$Duration>36 & Credit_Data$Duration<=48] <- 4
# Credit_Data$duration[Credit_Data$Duration>48 & Credit_Data$Duration<=60] <- 5
# Credit_Data$duration[Credit_Data$Duration>60 & Credit_Data$Duration<=72] <- 6
# Credit_Data$amount[Credit_Data$Credit.amount>0 & Credit_Data$Credit.amount<=1300] <- 1
# Credit_Data$amount[Credit_Data$Credit.amount>1300 & Credit_Data$Credit.amount<=3300] <- 2
# Credit_Data$amount[Credit_Data$Credit.amount>3300 & Credit_Data$Credit.amount<=4000] <- 3
# Credit_Data$amount[Credit_Data$Credit.amount>4000 & Credit_Data$Credit.amount<=20000] <- 4
Credit_Data$purpose[Credit_Data$Purpose == "radio/TV"] <- 1
Credit_Data$purpose[Credit_Data$Purpose == "education"] <- 2
Credit_Data$purpose[Credit_Data$Purpose == "car"] <- 3
Credit_Data$purpose[Credit_Data$Purpose == "business"] <- 4
Credit_Data$purpose[Credit_Data$Purpose == "furniture/equipment"] <- 5
Credit_Data$purpose[Credit_Data$Purpose == "vacation/others"] <- 6
Credit_Data$purpose[Credit_Data$Purpose == "repairs"] <- 7
Credit_Data$purpose[Credit_Data$Purpose == "domestic appliances"] <- 8
# Credit_Data$age[Credit_Data$Age>0 & Credit_Data$Age<=20] <- 1
# Credit_Data$age[Credit_Data$Age>20 & Credit_Data$Age<=40] <- 2
# Credit_Data$age[Credit_Data$Age>40 & Credit_Data$Age<=60] <- 3
# Credit_Data$age[Credit_Data$Age>60 & Credit_Data$Age<=80] <- 4
# Credit_Data$age[Credit_Data$Age>80 & Credit_Data$Age<=100] <- 5
Credit_Data

,X.,Age,Sex,Job,Housing,Saving.accounts,Checking.account,Credit.amount,Duration,Purpose,Credit.Risks,sex,housing,savings,checkings,purpose
2,1,22,female,2,own,little,moderate,5951,48,radio/TV,2,2,1,1,2,1
4,3,45,male,2,free,little,little,7882,42,furniture/equipment,1,1,2,1,1,5
5,4,53,male,2,free,little,little,4870,24,car,2,1,2,1,1,3
8,7,35,male,3,rent,little,moderate,6948,36,car,1,1,3,1,2,3
10,9,28,male,3,own,little,moderate,5234,30,car,2,1,1,1,2,3
11,10,25,female,2,rent,little,moderate,1295,12,car,2,2,3,1,2,3
12,11,24,female,2,rent,little,little,4308,48,business,2,2,3,1,1,4
13,12,22,female,2,own,little,moderate,1567,12,radio/TV,1,2,1,1,2,1
14,13,60,male,1,own,little,little,1199,24,car,2,1,1,1,1,3
15,14,28,female,2,rent,little,little,1403,15,car,1,2,3,1,1,3


In [6]:
#  normalize <- function(x) {
#      return ((x - min(x)) / (max(x) - min(x)))
#    }
cols<- sapply(Credit_Data,is.numeric)
cols
 Credit_Data[cols] <- as.data.frame(lapply(Credit_Data[cols], scale))

X.              Age              Sex              Job 
            TRUE             TRUE            FALSE             TRUE 
         Housing  Saving.accounts Checking.account    Credit.amount 
           FALSE            FALSE            FALSE             TRUE 
        Duration          Purpose     Credit.Risks              sex 
            TRUE            FALSE             TRUE             TRUE 
         housing          savings        checkings          purpose 
            TRUE             TRUE             TRUE             TRUE

Try Knn on the above data set for various Cases:

### Case 1: 80% Training Data - 20% Test Data

In [62]:
n.points <- 522 # number of rows in the Credit Data Table
sampling.rate <- 0.8
num.test.set.labels <- n.points * (1 - sampling.rate)
# randomly sample which rows will go in the training set
training <- sample(1:n.points, sampling.rate * n.points, replace=FALSE)
#train <- subset(Credit_Data[training, ], select = c(Age,factor(Sex),Job,factor(Housing),factor(Saving.accounts),factor(Checking.account),Credit.amount, Duration,factor(Purpose)))
#train <- subset(Credit_Data[training, ], select = c(Age,sex,Job,housing,savings,checkings,Credit.amount, Duration,purpose))
#train <- subset(Credit_Data[training, ], select = c(Age,factor(Saving.accounts),factor(Checking.account)))
train <- subset(Credit_Data[training, ], select = c(Age,sex,Job,housing,savings,checkings,Duration,Credit.amount,purpose))
#train <- subset(Credit_Data[training, ], select = c(Age,savings,checkings,Duration,Credit.amount))
#train <- subset(Credit_Data[training, ], select = c(age,sex,Job,housing,savings,checkings,duration,amount,purpose))
testing <- setdiff(1:n.points, training)
#test <- subset(Credit_Data[testing, ], select = c(Age,factor(Sex),Job,factor(Housing),factor(Saving.accounts),factor(Checking.account),Credit.amount, Duration,factor(Purpose)))
test <- subset(Credit_Data[testing, ], select = c(Age,sex,Job,housing,savings,checkings,Credit.amount, Duration,purpose))
#test <- subset(Credit_Data[testing, ], select = c(Age,savings,checkings,Duration,Credit.amount))
#test <- subset(Credit_Data[testing, ], select = c(Age,sex,Job,housing,savings,checkings,Duration,Credit.amount,purpose))
#test <- subset(Credit_Data[testing, ], select = c(Age,factor(Saving.accounts),factor(Checking.account)))
cl <- Credit_Data$Credit.Risks[training]
#str(cl)
#str(train)
summary(test$duration)
true.labels <- Credit_Data$Credit.Risks[testing]

Length  Class   Mode 
     0   NULL   NULL 

#### Test for k in 1-20 

In [63]:
df_80<- data.frame()
for (k in 1:20) {
    
print(k)
    predicted.labels <- knn(train, test, cl, k)
num.incorrect.labels <- sum(predicted.labels != true.labels)
misclassification.rate <- num.incorrect.labels /num.test.set.labels
print(misclassification.rate)
    df_80 <- rbind(df_80,data.frame(k,misclassification.rate))
}
df_80

[1] 1
[1] 0.3639847
[1] 2
[1] 0.3544061
[1] 3
[1] 0.3639847
[1] 4
[1] 0.3831418
[1] 5
[1] 0.3831418
[1] 6
[1] 0.2969349
[1] 7
[1] 0.335249
[1] 8
[1] 0.3256705
[1] 9
[1] 0.3927203
[1] 10
[1] 0.3065134
[1] 11
[1] 0.3448276
[1] 12
[1] 0.3065134
[1] 13
[1] 0.3256705
[1] 14
[1] 0.335249
[1] 15
[1] 0.335249
[1] 16
[1] 0.3256705
[1] 17
[1] 0.3256705
[1] 18
[1] 0.3448276
[1] 19
[1] 0.3639847
[1] 20
[1] 0.335249


k,misclassification.rate
1,0.3639847
2,0.3544061
3,0.3639847
4,0.3831418
5,0.3831418
6,0.2969349
7,0.3352490
8,0.3256705
9,0.3927203
10,0.3065134


### Case 2 - 70% Training Data - 30% Test Data

In [148]:
n.points <- 522 # number of rows in the dataset
sampling.rate <- 0.7
num.test.set.labels <- n.points * (1 - sampling.rate)
training <- sample(1:n.points, sampling.rate * n.points,
replace=FALSE)
#train <- subset(Credit_Data[training, ], select = c(Age,Sex,Job,Housing,Saving.accounts,Checking.account, Credit.amount, Duration,Purpose, Credit.Risks))
#train <- subset(Credit_Data[training, ], select = c(Age,savings,checkings))
#train <- subset(Credit_Data[training, ], select = c(Age,amount))
train <- subset(Credit_Data[training, ], select = c(Age,sex,Job,housing,savings,checkings,Duration,Credit.amount,purpose))
# define the training set to be those rows
testing <- setdiff(1:n.points, training)
# define the test set to be the other rows
#test <- subset(Credit_Data[testing, ], select = c(Age,Sex,Job,Housing,Saving.accounts,Checking.account, Credit.amount, Duration,Purpose, Credit.Risks))
test <- subset(Credit_Data[testing, ], select = c(Age,sex,Job,housing,savings,checkings,Duration,Credit.amount,purpose))
#test <- subset(Credit_Data[testing, ], select = c(Age,savings,checkings))
#test <- subset(Credit_Data[testing, ], select = c(Age,amount))
cl <- Credit_Data$Credit.Risks[training]
true.labels <- Credit_Data$Credit.Risks[testing]

#### Test for k in 1-20 

In [149]:
df_70 <- data.frame()
for (k in 1:20) {
    
print(k)
    predicted.labels <- knn(train, test, cl, k)
num.incorrect.labels <- sum(predicted.labels != true.labels)
misclassification.rate <- num.incorrect.labels /num.test.set.labels
print(misclassification.rate)
    df_70 <- rbind(df_70,data.frame(k,misclassification.rate))
}
df_70

[1] 1
[1] 0.4150702
[1] 2
[1] 0.4342273
[1] 3
[1] 0.4150702
[1] 4
[1] 0.4533844
[1] 5
[1] 0.4533844
[1] 6
[1] 0.4597701
[1] 7
[1] 0.4150702
[1] 8
[1] 0.4214559
[1] 9
[1] 0.4278416
[1] 10
[1] 0.4214559
[1] 11
[1] 0.3895275
[1] 12
[1] 0.3767561
[1] 13
[1] 0.3703704
[1] 14
[1] 0.3639847
[1] 15
[1] 0.357599
[1] 16
[1] 0.357599
[1] 17
[1] 0.3448276
[1] 18
[1] 0.3512133
[1] 19
[1] 0.3448276
[1] 20
[1] 0.3959132


k,misclassification.rate
1,0.4150702
2,0.4342273
3,0.4150702
4,0.4533844
5,0.4533844
6,0.4597701
7,0.4150702
8,0.4214559
9,0.4278416
10,0.4214559


### Case 3 - 90% Training Data - 10% Test Data 

In [176]:
n.points <- 522 # number of rows in the dataset
sampling.rate <- 0.9
# we need the number of points in the test set to calculate
# the misclassification rate
num.test.set.labels <- n.points * (1 - sampling.rate)
# randomly sample which rows will go in the training set
training <- sample(1:n.points, sampling.rate * n.points,
replace=FALSE)
#train <- subset(Credit_Data[training, ], select = c(Age,Sex,Job,Housing,Saving.accounts,Checking.account, Credit.amount, Duration,Purpose, Credit.Risks))
#train <- subset(Credit_Data[training, ], select = c(Age,savings,checkings,Credit.Risks))
train <- subset(Credit_Data[training, ], select = c(Age,sex,Job,housing,savings,checkings,Duration,Credit.amount,purpose))
# define the training set to be those rows
testing <- setdiff(1:n.points, training)
# define the test set to be the other rows
#test <- subset(Credit_Data[testing, ], select = c(Age,Sex,Job,Housing,Saving.accounts,Checking.account, Credit.amount, Duration,Purpose, Credit.Risks))
test <- subset(Credit_Data[testing, ], select = c(Age,sex,Job,housing,savings,checkings,Duration,Credit.amount,purpose))
#test <- subset(Credit_Data[testing, ], select = c(Age,savings,checkings,Credit.Risks))
cl <- Credit_Data$Credit.Risks[training]
# this is the subset of labels for the training set
true.labels <- Credit_Data$Credit.Risks[testing]
# subset of labels for the test set, we're withholding these

#### Test for k in 1-20 

In [177]:
df_90 <- data.frame()
for (k in 1:20) {
    
print(k)
    predicted.labels <- knn(train, test, cl, k)
num.incorrect.labels <- sum(predicted.labels != true.labels)
misclassification.rate <- num.incorrect.labels /num.test.set.labels
print(misclassification.rate)
    df_90 <- rbind(df_90,data.frame(k,misclassification.rate))
}
df_90

[1] 1
[1] 0.3831418
[1] 2
[1] 0.2873563
[1] 3
[1] 0.3065134
[1] 4
[1] 0.3639847
[1] 5
[1] 0.3256705
[1] 6
[1] 0.2873563
[1] 7
[1] 0.3065134
[1] 8
[1] 0.3448276
[1] 9
[1] 0.3639847
[1] 10
[1] 0.3448276
[1] 11
[1] 0.3065134
[1] 12
[1] 0.3448276
[1] 13
[1] 0.3065134
[1] 14
[1] 0.3256705
[1] 15
[1] 0.2490421
[1] 16
[1] 0.2681992
[1] 17
[1] 0.3065134
[1] 18
[1] 0.2873563
[1] 19
[1] 0.2681992
[1] 20
[1] 0.2873563


k,misclassification.rate
1,0.3831418
2,0.2873563
3,0.3065134
4,0.3639847
5,0.3256705
6,0.2873563
7,0.3065134
8,0.3448276
9,0.3639847
10,0.3448276


### Case 4 - 60% Training Data - 40% Test Data 

In [236]:
n.points <- 522
sampling.rate <- 0.6
num.test.set.labels <- n.points * (1 - sampling.rate)
training <- sample(1:n.points, sampling.rate * n.points,
replace=FALSE)
#train <- subset(Credit_Data[training, ], select = c(Age,Sex,Job,Housing,Saving.accounts,Checking.account, Credit.amount, Duration,Purpose, Credit.Risks))
#train <- subset(Credit_Data[training, ], select = c(Age,savings,checkings,Credit.Risks))
train <- subset(Credit_Data[training, ], select = c(Age,sex,Job,housing,savings,checkings,Duration,Credit.amount,purpose))
testing <- setdiff(1:n.points, training)
#test <- subset(Credit_Data[testing, ], select = c(Age,Sex,Job,Housing,Saving.accounts,Checking.account, Credit.amount, Duration,Purpose, Credit.Risks))
test <- subset(Credit_Data[testing, ], select = c(Age,sex,Job,housing,savings,checkings,Duration,Credit.amount,purpose))
#test <- subset(Credit_Data[testing, ], select = c(Age,savings,checkings,Credit.Risks))
cl <- Credit_Data$Credit.Risks[training]
true.labels <- Credit_Data$Credit.Risks[testing]

#### Test for k in 1-20 

In [237]:
df_60 <- data.frame()
for (k in 1:20) {
print(k)
    predicted.labels <- knn(train, test, cl, k)
num.incorrect.labels <- sum(predicted.labels != true.labels)
misclassification.rate <- num.incorrect.labels /num.test.set.labels
print(misclassification.rate)
    df_60 <- rbind(df_60,data.frame(k,misclassification.rate))
}
df_60

[1] 1
[1] 0.4022989
[1] 2
[1] 0.3544061
[1] 3
[1] 0.3927203
[1] 4
[1] 0.4214559
[1] 5
[1] 0.4022989
[1] 6
[1] 0.3927203
[1] 7
[1] 0.4118774
[1] 8
[1] 0.3927203
[1] 9
[1] 0.3927203
[1] 10
[1] 0.4214559
[1] 11
[1] 0.4118774
[1] 12
[1] 0.440613
[1] 13
[1] 0.4214559
[1] 14
[1] 0.4358238
[1] 15
[1] 0.4214559
[1] 16
[1] 0.4166667
[1] 17
[1] 0.3975096
[1] 18
[1] 0.3783525
[1] 19
[1] 0.3927203
[1] 20
[1] 0.3639847


k,misclassification.rate
1,0.4022989
2,0.3544061
3,0.3927203
4,0.4214559
5,0.4022989
6,0.3927203
7,0.4118774
8,0.3927203
9,0.3927203
10,0.4214559


#### Merge the Tables

In [253]:
Table <- NULL
#df_90
#df_80
#df_70
Table <- merge(df_90,df_80,by="k")
Table <- merge(Table,df_70,by="k")
#Table
Table <- rename.vars(Table,c("misclassification.rate.x","misclassification.rate.y","misclassification.rate"),c("Sampling Rate=90","Sampling Rate=80","Sampling Rate=70"))
Table <- merge(Table,df_60,by="k")
Table <- rename.vars(Table,c("misclassification.rate"),c("Sampling Rate=60"))
Table


Changing in Table                                                                              
From: misclassification.rate.x misclassification.rate.y misclassification.rate
To:   Sampling Rate=90         Sampling Rate=80         Sampling Rate=70      


Changing in Table                            
From: misclassification.rate
To:   Sampling Rate=60      



k,Sampling Rate=90,Sampling Rate=80,Sampling Rate=70,Sampling Rate=60
1,0.3831418,0.3639847,0.4150702,0.4022989
2,0.2873563,0.3544061,0.4342273,0.3544061
3,0.3065134,0.3639847,0.4150702,0.3927203
4,0.3639847,0.3831418,0.4533844,0.4214559
5,0.3256705,0.3831418,0.4533844,0.4022989
6,0.2873563,0.2969349,0.4597701,0.3927203
7,0.3065134,0.3352490,0.4150702,0.4118774
8,0.3448276,0.3256705,0.4214559,0.3927203
9,0.3639847,0.3927203,0.4278416,0.3927203
10,0.3448276,0.3065134,0.4214559,0.4214559


## Observation:
### Lowest Value for Sampling Rate=90: 0.2490421 (k=15)
### Lowest Value for Sampling Rate=80: 0.2969349 (k=6)
### Lowest Value for Sampling Rate=70: 0.3448276 (k=17 & k=19)
### Lowest Value for Sampling Rate=60: 0.3544061 (k=2)

In [256]:
#write.csv(Table,"knnresult.csv",row.names=FALSE)